```{autolink-concat}
```

::::{margin}
:::{card} Definition of the normalized Blatt–Weisskopf form factor from Hankel functions of the first kind.
TR-029
^^^
This report investigates how to implement [ComPWA/ampform#417](https://github.com/ComPWA/ampform/issues/417), where it was suggested to define the 'normalized' Blatt–Weisskopf function $B_L^2(z)$ from a Hankel function of the first kind, $h_l^{(1)}$.
:::
::::

# Blatt–Weisskopf from Hankel function

In [ ]:
%pip install -q ampform==0.15.1 sympy==1.12

In [ ]:
from __future__ import annotations

from functools import cache

import sympy as sp
from ampform.dynamics.phasespace import BreakupMomentumSquared
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math, display

## Introduction

As of AmpForm [v0.15](https://github.com/ComPWA/ampform/releases/tag/0.15.1), the implementation of [`BlattWeisskopfSquared`](https://ampform.readthedocs.io/0.15.x/api/ampform.dynamics/#ampform.dynamics.BlattWeisskopfSquared) contains hard-coded polynomials, see implementation [here](https://github.com/ComPWA/ampform/blob/0.15.1/src/ampform/dynamics/__init__.py#L66-L134).
The motivation for this can be found in the citations mentioned in [its API documentation](https://ampform.readthedocs.io/0.15.x/api/ampform.dynamics/#ampform.dynamics.BlattWeisskopfSquared).
However, as noted by [@mmikhasenko](https://github.com/mmikhasenko) in [ComPWA/ampform#417](https://github.com/ComPWA/ampform/issues/417), the polynomials can be derived from the spherical[^1] Hankel functions of the first kind.
Von Hippel and Quigg[^2] derived a generalization of the centrifugal barrier factor&nbsp;$F_L$, also called form factor, that was introduced by {cite}`Blatt:1952ije`, showing that

[^1]: See [this page](https://mathworld.wolfram.com/SphericalHankelFunctionoftheFirstKind.html) on Wolfram MathWorld for an explanation about the difference between $h_\ell^{(1)}$ and $H_\ell^{(1)}$.
[^2]: See {cite}`VonHippel:1972fg`, pp.&nbsp;626 and 637, and a review by COMPASS, {cite}`Ketzer:2019wmd`, p.&nbsp;31.

```{math}
F_\ell^2(z^2) = \frac{1}{z^2\left|h^{(1)}_\ell\left(z\right)\right|^2}\,,
```

where $h_\ell^{(1)}$ is a Hankel function of the first kind. They also noted that, if $z\in\mathbb{R}$,

$$
h_\ell^{(1)}(z) =
  \left(- i\right)^{\ell+1}
  \frac{e^{iz}}{z}
  \sum_{k=0}^\ell
    \frac{(\ell+k)!}{(\ell-k)! \, k!}
    \left(\frac{i}{2z}\right)^k.
$$ (hankel-sum)

In the following, we call $F_\ell(z)$ the _unnormalized_ Blatt–Weisskopf form factor.
Following Chung and other resources (see e.g. {cite}`Chung:1995dx`, p.&nbsp;415), AmpForm implements a unitless, _normalized_ Blatt–Weisskopf factor $B_L$, meaning that $B_L(1)=1$.[^3]
It can be defined in terms of $F_L$ as

[^3]: We switch to notating angular momentum with $L$ instead of $\ell$ here to indicate that we are talking about a normalized function here.

```{math}
B_L^2(z^2)
  = \frac{F_L^2(z^2)}{F_L^2(1)}
  = \frac{\left|h^{(1)}_L(1)\right|^2}{z^2\left|h^{(1)}_L(z)\right|^2}\,.
```

:::{note}
As of writing, AmpForm uses $z$ as argument in [`BlattWeisskopfSquared`](https://ampform.readthedocs.io/0.15.x/api/ampform.dynamics/#ampform.dynamics.BlattWeisskopfSquared).
This means we have to work with a square root and assume that $z \geq 0$, meaning

$$
B_L^2(z) = \frac{\left|h^{(1)}_L(1)\right|^2}{z\left|h^{(1)}_L\left(\sqrt{z}\right)\right|^2}\,.
$$ (blatt-weisskopf)

:::

## Hankel function of the first kind

### Built-in SymPy function

SymPy offers a Hankel function of the first kind, [`scipy.special.hankel1`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.hankel1.html).

In [ ]:
z = sp.Symbol("z", nonnegative=True, real=True)
ell = sp.Symbol(R"\ell", integer=True, nonnegative=True)
sp.hankel1(ell, z)

This function is the general[^1] Hankel function $H_\ell$ and the class does not offer algebraic simplifications for specific values or assumptions of $\ell$ and $z$.

In [ ]:
display(
    sp.hankel1(ell, z).doit(),
    sp.hankel1(ell, 0).doit(),
    sp.hankel1(0, z).doit(),
    sp.hankel1(0, 0).doit(),
)

### Custom class definition

To implement Equation&nbsp;{eq}`hankel-sum` for the _spherical_ Hankel function, we have to define a custom [`@unevaluated`](https://ampform.readthedocs.io/0.15.x/api/ampform.sympy/#ampform.sympy.unevaluated) expression class.
The following class evaluates to the sum given in Equation&nbsp;{eq}`hankel-sum`.
We introduce a special [`sympy.Sum`](https://docs.sympy.org/latest/modules/concrete.html#sympy.concrete.summations.Sum) class that does not 'unfold' on symbolic input for $\ell$ if [`doit()`](https://docs.sympy.org/latest/modules/core.html#sympy.core.basic.Basic.doit) is called (see [](#nested-doit)).

In [ ]:
@unevaluated
class SphericalHankel1(sp.Expr):
    l: sp.Symbol | int
    z: sp.Symbol | float
    _latex_repr_ = R"h_{{{l}}}^{{(1)}}\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        l, z = self.args
        k = sp.Dummy("k", integer=True, nonnegative=True)
        return (
            (-sp.I) ** (1 + l)
            * (sp.exp(z * sp.I) / z)
            * SymbolicSum(
                sp.factorial(l + k)
                / (sp.factorial(l - k) * sp.factorial(k))
                * (sp.I / (2 * z)) ** k,
                (k, 0, l),
            )
        )


class SymbolicSum(sp.Sum):
    def doit(self, deep: bool = True, **kwargs) -> sp.Expr:
        if _get_indices(self):
            expression = self.args[0]
            indices = self.args[1:]
            return SymbolicSum(expression.doit(deep=deep, **kwargs), *indices)
        return super().doit(deep=deep, **kwargs)


@cache
def _get_indices(expr: sp.Sum) -> set[sp.Symbol]:
    free_symbols = set()
    for index in expr.args[1:]:
        free_symbols.update(index.free_symbols)
    return {s for s in free_symbols if not isinstance(s, sp.Dummy)}

In [ ]:
h1lz = SphericalHankel1(ell, z)
Math(aslatex({h1lz: h1lz.doit()}))

Indeed, the absolute squared value $\left|h_\ell^{(1)}\right|^2$ results in a clean fraction of polynomials (after some algebraic [simplifications](https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html)).

In [ ]:
exprs = [sp.Abs(h1lz.xreplace({ell: i})) ** 2 for i in range(3)]
Math(aslatex({e: e.doit().simplify() for e in exprs}))

In [ ]:
exprs = [sp.Abs(h1lz.xreplace({ell: i, z: 1})) ** 2 for i in range(3)]
Math(aslatex({e: e.doit() for e in exprs}))

## Normalized Blatt–Weisskopf form factor

We now have the required expression classes for re-implementing [`BlattWeisskopfSquared`](https://ampform.readthedocs.io/0.15.x/api/ampform.dynamics/#ampform.dynamics.BlattWeisskopfSquared) using Equation&nbsp;{eq}`blatt-weisskopf` (with $z$ as input, instead of $z^2$).

In [ ]:
@unevaluated
class BlattWeisskopfSquared(sp.Expr):
    L: sp.Symbol | int
    z: sp.Symbol | float
    _latex_repr_ = R"B^2_{{{L}}}\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        L = self.L
        z = sp.Dummy("z", nonnegative=True, real=True)
        expr = (
            sp.Abs(SphericalHankel1(L, 1)) ** 2
            / sp.Abs(SphericalHankel1(L, sp.sqrt(z))) ** 2
            / z
        )
        if not L.free_symbols:
            expr = expr.doit().simplify()
        return expr.xreplace({z: self.z})

:::{note}
An explicit [`simplify()`](https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#simplify) is required in order to reproduce the polynomial form upon evaluation.
To make the simplification as fast as possible, it is done internally within `evaluate()` with $z$ as a dummy variable.
This is to avoid performing nested simplifications if $z$ is in itself an expression (see [](#nested-doit)).
:::

In [ ]:
L = sp.Symbol("L", integer=True, nonnegative=True)
BL2 = BlattWeisskopfSquared(L, z)
Math(aslatex({BL2: BL2.doit(deep=False)}))

Indeed the polynomials are exactly the same as the [original `BlattWeisskopfSquared`](https://ampform.readthedocs.io/0.15.x/api/ampform.dynamics/#ampform.dynamics.BlattWeisskopfSquared)!

In [ ]:
exprs = [BL2.xreplace({L: i}) for i in range(9)]
Math(aslatex({e: e.doit() for e in exprs}))

## Nested doit

Eventually, the barrier factors take $z=q/q_R$, with $q$ the break-up momentum and $q_R$ an impact factor. Here it becomes crucial that only $\left|h_\ell^{(1)}(z)\right|^2$ is simplified to a polynomial fraction, not $q$ itself. The break-up momentum does need to unfold though.

In [ ]:
s, m1, m2, qR = sp.symbols("s m1 m2 q_R", nonnegative=True)
q2 = BreakupMomentumSquared(s, m1, m2)
Math(aslatex({q2: q2.doit()}))

### Symbolic angular momentum

In [ ]:
BlattWeisskopfSquared(L, z=q2 / qR**2).doit(deep=False)

In [ ]:
BlattWeisskopfSquared(L, z=q2 / qR**2).doit(deep=True)

### Numeric angular momentum

In [ ]:
BlattWeisskopfSquared(L=2, z=q2 / qR**2).doit(deep=False)

In [ ]:
BlattWeisskopfSquared(L=2, z=q2 / qR**2).doit(deep=True)